In [1]:
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path_DF = '/content/drive/My Drive/master project/data/DF4'

Mounted at /content/drive


In [2]:
import lightgbm
import sklearn
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

In [3]:
df_META = pd.read_csv(os.path.join(path_DF, 'df_META.csv'))
df_AAPL= pd.read_csv(os.path.join(path_DF, 'df_AAPL.csv'))
df_MSFT= pd.read_csv(os.path.join(path_DF, 'df_MSFT.csv'))
df_NVDA= pd.read_csv(os.path.join(path_DF, 'df_NVDA.csv'))
df_GOOG= pd.read_csv(os.path.join(path_DF, 'df_GOOG.csv'))

In [4]:
df_META=df_META.drop(['Unnamed: 0'], axis=1)
df_META['date'] = pd.to_datetime(df_META['date'])
df_META = df_META.sort_values('date').reset_index(drop=True)

In [5]:
df_AAPL=df_AAPL.drop(['Unnamed: 0'], axis=1)
df_AAPL['date'] = pd.to_datetime(df_AAPL['date'])
df_AAPL = df_AAPL.sort_values('date').reset_index(drop=True)

In [6]:
df_MSFT=df_MSFT.drop(['Unnamed: 0'], axis=1)
df_MSFT['date'] = pd.to_datetime(df_MSFT['date'])
df_MSFT = df_MSFT.sort_values('date').reset_index(drop=True)

In [7]:
df_NVDA=df_NVDA.drop(['Unnamed: 0'], axis=1)
df_NVDA['date'] = pd.to_datetime(df_NVDA['date'])
df_NVDA = df_NVDA.sort_values('date').reset_index(drop=True)

In [8]:
df_GOOG=df_GOOG.drop(['Unnamed: 0'], axis=1)
df_GOOG['date'] = pd.to_datetime(df_GOOG['date'])
df_GOOG = df_GOOG.sort_values('date').reset_index(drop=True)

In [9]:
train_size = int(len(df_META) * 0.75)  # 75% ของ 753
test_size = len(df_META) - train_size  # 25% ที่เหลือ

In [10]:
print(f"Train size: {train_size}")
print(f"Test size: {test_size}")

Train size: 564
Test size: 189


# META

## Train Test

In [11]:
train_META = df_META.iloc[:train_size]  # ข้อมูล train
test_META = df_META.iloc[train_size:]  # ข้อมูล test

In [12]:
train_META

,date,Stockprice_META,sentiment_META,Interest,UNEMPLOYMENT,CPI,US_Treasury_Yield,RETAIL_SALES,US_inflation_rate,Real_GDP,US_Dollar_Index,S&P500,NASDAQ,WTI,BTC,Gold
0,2022-03-01,203.49,0.000000,0.08,3.7,287.504,1.72,595064.0,0.075,5337.277,97.41,4306.26,13532.46,107.77,44420.3,1943.8
1,2022-03-02,208.11,0.000000,0.08,3.7,287.504,1.86,595064.0,0.075,5337.277,97.39,4386.54,13752.02,112.22,43912.8,1922.3
2,2022-03-03,202.97,0.000000,0.08,3.7,287.504,1.86,595064.0,0.075,5337.277,97.79,4363.49,13537.94,110.52,42463.0,1935.9
3,2022-03-04,200.06,0.000000,0.08,3.7,287.504,1.74,595064.0,0.075,5337.277,98.65,4328.87,13313.44,116.37,39142.7,1966.6
4,2022-03-07,187.47,0.000000,0.08,3.7,287.504,1.78,595064.0,0.075,5337.277,99.29,4201.09,12830.96,122.09,38024.8,1995.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,2024-05-21,464.63,0.129250,5.33,4.0,314.069,4.41,643843.0,0.034,5817.169,104.66,5321.41,16832.62,78.69,70139.9,2425.9
560,2024-05-22,467.78,0.622400,5.33,4.0,314.069,4.43,643843.0,0.034,5817.169,104.93,5307.01,16801.54,77.53,69155.4,2392.9
561,2024-05-23,465.78,0.399033,5.33,4.0,314.069,4.47,643843.0,0.034,5817.169,105.11,5267.84,16736.03,77.22,67975.7,2337.2
562,2024-05-24,478.22,0.437117,5.33,4.0,314.069,4.46,643843.0,0.034,5817.169,104.72,5304.72,16920.79,78.18,68547.6,2334.5


### Train Varidation

In [13]:
X_META = train_META.drop(columns=["Stockprice_META","date","sentiment_META","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_META = train_META["Stockprice_META"]

In [14]:
X_META

,Interest,UNEMPLOYMENT,CPI,US_Treasury_Yield,US_inflation_rate,Real_GDP,S&P500,NASDAQ,BTC,Gold
0,0.08,3.7,287.504,1.72,0.075,5337.277,4306.26,13532.46,44420.3,1943.8
1,0.08,3.7,287.504,1.86,0.075,5337.277,4386.54,13752.02,43912.8,1922.3
2,0.08,3.7,287.504,1.86,0.075,5337.277,4363.49,13537.94,42463.0,1935.9
3,0.08,3.7,287.504,1.74,0.075,5337.277,4328.87,13313.44,39142.7,1966.6
4,0.08,3.7,287.504,1.78,0.075,5337.277,4201.09,12830.96,38024.8,1995.9
...,...,...,...,...,...,...,...,...,...,...
559,5.33,4.0,314.069,4.41,0.034,5817.169,5321.41,16832.62,70139.9,2425.9
560,5.33,4.0,314.069,4.43,0.034,5817.169,5307.01,16801.54,69155.4,2392.9
561,5.33,4.0,314.069,4.47,0.034,5817.169,5267.84,16736.03,67975.7,2337.2
562,5.33,4.0,314.069,4.46,0.034,5817.169,5304.72,16920.79,68547.6,2334.5


#Normalized

In [15]:
scaler = MinMaxScaler()

X_META = scaler.fit_transform(X_META)

In [17]:
model = LGBMRegressor(random_state=42, verbosity=-1)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv,
                           scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_META , y_META)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_META)
mse = mean_squared_error(y_META, y_pred)
rmse = root_mean_squared_error(y_META, y_pred)
mae = mean_absolute_error(y_META, y_pred)
mape = mean_absolute_percentage_error(y_META, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000148 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 329
[LightGBM] [Info] Number of data points in the train set: 564, number of used features: 10
[LightGBM] [Info] Start training from score 260.227846
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warni

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#AAPL

In [18]:
train_AAPL = df_AAPL.iloc[:train_size]  # ข้อมูล train
test_AAPL = df_AAPL.iloc[train_size:]  # ข้อมูล test

In [19]:
X_AAPL = train_AAPL.drop(columns=["Stockprice_AAPL","sentiment_AAPL", "date", "Interest", "US_Treasury_Yield", "RETAIL_SALES", "US_Dollar_Index"])
y_AAPL = train_AAPL["Stockprice_AAPL"]

In [20]:
X_AAPL = scaler.fit_transform(X_AAPL)

In [21]:
model = LGBMRegressor(random_state=42, verbosity=-1)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv,
                           scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_AAPL , y_AAPL)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_AAPL)
mse = mean_squared_error(y_AAPL, y_pred)
rmse = root_mean_squared_error(y_AAPL, y_pred)
mae = mean_absolute_error(y_AAPL, y_pred)
mape = mean_absolute_percentage_error(y_AAPL, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1005
[LightGBM] [Info] Number of data points in the train set: 564, number of used features: 9
[LightGBM] [Info] Start training from score 166.177748
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warni

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#MSFT

In [22]:
train_MSFT = df_MSFT.iloc[:train_size]  # ข้อมูล train
test_MSFT = df_MSFT.iloc[train_size:]  # ข้อมูล test

In [23]:
X_MSFT = train_MSFT.drop(columns=["Stockprice_MSFT","sentiment_MSFT","date","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_MSFT = train_MSFT["Stockprice_MSFT"]

In [24]:
X_MSFT = scaler.fit_transform(X_MSFT)

In [25]:
model = LGBMRegressor(random_state=42, verbosity=-1)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv,
                           scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_MSFT , y_MSFT)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_MSFT)
mse = mean_squared_error(y_MSFT, y_pred)
rmse = root_mean_squared_error(y_MSFT, y_pred)
mae = mean_absolute_error(y_MSFT, y_pred)
mape = mean_absolute_percentage_error(y_MSFT, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000118 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 329
[LightGBM] [Info] Number of data points in the train set: 564, number of used features: 10
[LightGBM] [Info] Start training from score 311.787261
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warni

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#NVDA

In [26]:
train_NVDA = df_NVDA.iloc[:train_size]  # ข้อมูล train
test_NVDA = df_NVDA.iloc[train_size:]  # ข้อมูล test

In [27]:
X_NVDA = train_NVDA.drop(columns=["Stockprice_NVDA","sentiment_NVDA","date", "Interest","RETAIL_SALES", "US_Dollar_Index"])
y_NVDA = train_NVDA["Stockprice_NVDA"]

In [28]:
X_NVDA = scaler.fit_transform(X_NVDA)

In [29]:
model = LGBMRegressor(random_state=42, verbosity=-1)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv,
                           scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_NVDA , y_NVDA)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_NVDA)
mse = mean_squared_error(y_NVDA, y_pred)
rmse = root_mean_squared_error(y_NVDA, y_pred)
mae = mean_absolute_error(y_NVDA, y_pred)
mape = mean_absolute_percentage_error(y_NVDA, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000061 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 665
[LightGBM] [Info] Number of data points in the train set: 564, number of used features: 10
[LightGBM] [Info] Start training from score 37.071705
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#GOOG

In [30]:
train_GOOG = df_GOOG.iloc[:train_size]  # ข้อมูล train
test_GOOG = df_GOOG.iloc[train_size:]  # ข้อมูล test

In [31]:
X_GOOG = train_GOOG.drop(columns=["Stockprice_GOOGL", "sentiment_GOOGL", "date", "Interest", "RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_GOOG = train_GOOG["Stockprice_GOOGL"]

In [32]:
X_GOOG = scaler.fit_transform(X_GOOG)

In [33]:
model = LGBMRegressor(random_state=42, verbosity=-1)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv,
                           scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_GOOG , y_GOOG)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_GOOG)
mse = mean_squared_error(y_GOOG, y_pred)
rmse = root_mean_squared_error(y_GOOG, y_pred)
mae = mean_absolute_error(y_GOOG, y_pred)
mape = mean_absolute_percentage_error(y_GOOG, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000119 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 315
[LightGBM] [Info] Number of data points in the train set: 564, number of used features: 9
[LightGBM] [Info] Start training from score 121.692454
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin